In [1]:
import pandas as pd
import numpy as np
import requests
import time
import json

from config import api_key
import pprint

import datetime as dt
from datetime import date, timedelta

In [2]:
# generate start and end dates for the whole year
start_date = '2019-01-01'
end_date = '2019-12-31'

# Use the start and end date to create a range of dates
start_dates = pd.date_range(start_date, end_date, freq='MS')
start_dates = list(np.ravel(pd.to_datetime(start_dates).strftime("%Y-%m-%d")))

end_dates = pd.date_range(end = end_date, periods = 12, freq='M')
end_dates = list(np.ravel(pd.to_datetime(end_dates).strftime("%Y-%m-%d")))


In [3]:
cities = ['Toronto', 'Vancouver', 'Boston', 'New+York']
city_id=0
location_id=[]
forecast_date = []
maxTempC = []
humidity = []
total_precip = []
avg_cloudcover = []
avg_windspeed = []

for n in range(len(cities)):
    city_name=cities[n]
    # loop thru all months in 2019
    url = f"http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key="
    city = f"&q={city_name}&format=json&date="
    city_id+=1
    
    for dt in range(len(start_dates)):
        query_url = f"{url}{api_key}{city}{start_dates[dt]}&enddate={end_dates[dt]}"
        

#request for data
        response=requests.get(query_url)
        weather_json=response.json()

# loop through one month data
        days = len(weather_json['data']['weather'])
        for day in range(days):
            dates = weather_json['data']['weather'][day]['date'] # date
            maxtempc = weather_json['data']['weather'][day]['maxtempC'] # maxTempC

#       append to lists
            forecast_date.append(dates)
            maxTempC.append(maxtempc)

            windspeed = 0
            totalprecip = 0
            cloud = 0
            humid = 0
            x = len(weather_json['data']['weather'][day]['hourly'])
            for i in range(x):
                windspeed += int(weather_json['data']['weather'][day]['hourly'][i]['windspeedKmph'])
                totalprecip += float(weather_json['data']['weather'][day]['hourly'][i]['precipMM'])
                cloud += int(weather_json['data']['weather'][day]['hourly'][i]['cloudcover'])
                humid += int(weather_json['data']['weather'][day]['hourly'][i]['humidity'])

            avg_humidity = humid/x
            avg_cloud = cloud/x
            avg_wind_speed = windspeed/x

# append weather info to lists
            humidity.append(round(avg_humidity,3))
            total_precip.append(round(totalprecip,3))
            avg_cloudcover.append(round(avg_cloud,3))
            avg_windspeed.append(round(avg_wind_speed,3))
            location_id.append(city_id)


In [8]:
weather_df = pd.DataFrame({"location_id": location_id, "forecast_date": forecast_date,
                           "maxtempc": maxTempC, "humidity": humidity,
                           "total_precip": total_precip, "avg_cloudcover": avg_cloudcover,
                           "avg_windspeed": avg_windspeed
                          })


In [9]:
weather_df

location_id forecast_date maxtempc  humidity  total_precip  \
0               1    2019-01-01        1    77.250           3.6   
1               1    2019-01-02       -5    69.000           0.2   
2               1    2019-01-03       -1    68.500           1.3   
3               1    2019-01-04        2    77.250           0.0   
4               1    2019-01-05        3    81.125           0.0   
...           ...           ...      ...       ...           ...   
1455            4    2019-12-27        9    88.750           0.5   
1456            4    2019-12-28       10    72.000           0.0   
1457            4    2019-12-29        7    64.000           1.3   
1458            4    2019-12-30        8    93.125          50.8   
1459            4    2019-12-31        8    71.500           0.1   

      avg_cloudcover  avg_windspeed  
0             50.625         16.875  
1             41.750          8.625  
2             68.250         17.750  
3              7.625         18.125  
4             23.750         11.250  
...              ...            ...  
1455          78.750          9.750  
1456          20.000         10.375  
1457          46.875          8.000  
1458          99.875         21.000  
1459          63.750         17.125  

[1460 rows x 7 columns]

In [10]:
weather_df.to_csv('historical_weather.csv', index=False)